**Segmentation d'images dans les documents historiques**


**Script d'auto apprentissage**

Ce script permet d'entraîner l'algorithme Mask-RCNN à la segmentation d'objets sur ses propres données d'entraînement. 
Il requiert un corpus d'entraînement : un fichier zip composé de deux dossiers "train" et "val" contenant chacun les images et leur fichier d'annotation nommé *via_region_data.json*

**Ce script doit impérativement être lancé dans un environnement GPU : Runtime -> Change runtime type -> GPU**

**1. Téléchargement de l'algorithme et installation des pré-requis**

In [ ]:
import os
%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  !pip install -r requirements.txt
  !python setup.py install
  !cp ~/Mask_RCNN/samples/balloon/balloon.py ./illustration.py
  !sed -i -- 's/balloon/illustration/g' illustration.py
  !sed -i -- 's/Balloon/Illustration/g' illustration.py
# !sed -i -- 's/epochs=30/epochs=40/g' illustration.py # Spécifier le nombre d'époques ici

  if not os.path.exists("/root/Mask_RCNN/weights"):
    os.makedirs('weights')
  if not os.path.exists("/root/Mask_RCNN/dataset"):
    os.makedirs('dataset')
else : print("L'algorithme est déjà téléchargé")

**2. Spécification des versions de TensorFlow et Keras**

In [ ]:
%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5

**3. Importation du corpus d'entraînement**

- **Pour un corpus annoté avec VIA :**

Importer un fichier zip constitué de cette architecture :

(le nom et le format des images sont libres)

(le fichier via_region_data.json est obtenu dans VIA par Annotation -> Export Annotations (as json)


```
  train
  img1.jpg
  img2.jpg
  ...
  img100.jpg
  via_region_data.json
val
  img101.jpg
  img102.jpg
  ...
  img120.jpg
  via_region_data.json
```

- **Pour un corpus annoté avec Annotate :**

un fichier zip constitué de cette architecture:

Le nom et le format des images sont libres

Le fichier annotate_region_data.csv est obtenu dans Annotate par View and Export results of annotation -> Annotations -> Export CSV -> Use Coma Separator

Testé avec Annotate 1.7



```
  train
  img1.jpg
  img2.jpg
  ...
  img100.jpg
  annotate_region_data.csv
val
  img101.jpg
  img102.jpg
  ...
  img120.jpg
  annotate_region_data.csv
```
**Trois possibilités au choix pour importer le corpus :**

1.   Importer le corpus d'entraînement fourni sur le github modoap-seg (entraîné sur 120 pages de la revue Droits Et Libertés (noir et blanc), et 30 époques)
2.   Importer son fichier Zip depuis son Google Drive (nécessite une authentification Google et le partage de son fichier) - téléchargement rapide
3.   Importer son fichier Zip depuis son disque local - téléchargement plus long

**3.1 Importer le corpus d'entraînement fourni sur le github modoap-seg**

In [ ]:
%cd /root/Mask_RCNN/dataset/
!wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/exemple_entrainement_droitsetlibertes/corpus_train_val.7z.001
!wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/exemple_entrainement_droitsetlibertes/corpus_train_val.7z.002
!wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/exemple_entrainement_droitsetlibertes/corpus_train_val.7z.003

!7z x /root/Mask_RCNN/dataset/corpus_train_val.7z.001
os.remove("/root/Mask_RCNN/dataset/corpus_train_val.7z.001")
os.remove("/root/Mask_RCNN/dataset/corpus_train_val.7z.002")
os.remove("/root/Mask_RCNN/dataset/corpus_train_val.7z.003")

!7z x /root/Mask_RCNN/dataset/corpus_entrainement_DL.zip
os.remove("/root/Mask_RCNN/dataset/corpus_entrainement_DL.zip")

**3.2 Importer un fichier Zip depuis son Google Drive**

Cette méthode nécessite de copier le lien partageable de son fichier sur son Google Drive (exemple : 1nWoZIRsw9Kx7CriJ3OgB436PPllV7veC), et de se connecter à son compte via le code de vérification fourni par Google.

In [ ]:
%cd /root/Mask_RCNN/dataset
fileId = input("Entrez le lien partageable du fichier ")

from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Fichier extrait ' + fileName)

**3.3 Importer un fichier zip depuis son disque local**

Lancer la cellule et cliquer sur "Browse". Le téléchargement est plus long.

In [ ]:
%cd /root/Mask_RCNN/dataset
from google.colab import files
corpus = files.upload()
fichier = list(corpus.keys())[0]
!7z x $fichier 
os.remove(fichier)

**3.4 Traduction des annotations réalisées avec VIA** (nécessaire)

Rappel : les fichiers d'annotations VIA dans les dossiers "train" et "val" doivent tous deux être nommés "via_region_data.json"

Cette cellule transforme les annotations rectangulaires en polygones à 4 coordonnées, traitables par l'algorithme

In [ ]:
import os
import json


def traductionVIA(viajson):
  annotations = json.load(open(viajson))
  if "_via_settings" in annotations.keys() :
      annotations_new = {}
      for image in annotations["_via_img_metadata"]:
          valeur_image = annotations["_via_img_metadata"][image]
          nom = image
          annotations_new[nom] = valeur_image
          for i in range(len(annotations["_via_img_metadata"][image]["regions"])) :
              if annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["name"] == "rect":

                  x = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["x"]
                  y = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["y"]
                  width = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["width"]
                  height = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["height"]

                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_x"] = [x, x+width, x+width, x]
                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_y"] = [y, y, y+height, y+height]
                  annotations_new[image]["regions"][i]["shape_attributes"]["name"] = "polygon"
                  del annotations_new[image]["regions"][i]["shape_attributes"]["y"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["x"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["width"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["height"]

  else :
      annotations_new = annotations
      for image in annotations :
          for i in range(len(annotations[image]["regions"])) :
              if annotations[image]["regions"][i]["shape_attributes"]["name"] == "rect":
                  x = annotations[image]["regions"][i]["shape_attributes"]["x"]
                  y = annotations[image]["regions"][i]["shape_attributes"]["y"]
                  width = annotations[image]["regions"][i]["shape_attributes"]["width"]
                  height = annotations[image]["regions"][i]["shape_attributes"]["height"]

                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_x"] = [x, x+width, x+width, x]
                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_y"] = [y, y, y+height, y+height]
                  annotations_new[image]["regions"][i]["shape_attributes"]["name"] = "polygon"
                  del annotations_new[image]["regions"][i]["shape_attributes"]["y"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["x"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["width"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["height"]
              
  with open('via_region_data.json', 'w') as json_file:
    json.dump(annotations_new, json_file)
    
%cd /root/Mask_RCNN/dataset/train
traductionVIA("via_region_data.json")

%cd /root/Mask_RCNN/dataset/val
traductionVIA("via_region_data.json")

**3.5 Traduction des annotations réalisées avec Annotate** (necessaire)

Rappel : les fichiers d'annotations Annotate dans les dossiers "train" et "val" doivent tous deux être nommés "annotate_region_data.csv"

In [ ]:
import os 
import pandas as pd

def traductionANNOTATE(annotations_csv):

  annotationz_new = {}
  annotationz = pd.read_csv(annotations_csv).sort_values('File')

  for index, row in annotationz.iterrows() :
    filename = row["File"]
    size = ""
    regions = {}
    x = int(row["X"])
    y = int(row["Y"])
    w = int(row["W"])
    h = int(row["H"])
    all_points_x = [x, x+w, x+w, x]
    all_points_y = [y, y, y+h, y+h]
      
    if filename not in annotationz_new.keys(): 
      annotationz_new[filename] = {"filename":filename, "size":size,"regions":{0:{"shape_attributes":{"name":"polygon", "all_points_x":all_points_x,"all_points_y":all_points_y}, "region_attributes":{}}}, "file_attributes":{}}
      
    else :
      i = max(annotationz_new[filename]["regions"].keys()) + 1
      annotationz_new[filename]["regions"][i] = {"shape_attributes":{"name":"polygon", "all_points_x":all_points_x,"all_points_y":all_points_y}, "region_attributes":{}}

  with open('via_region_data.json', 'w') as json_file:
    json.dump(annotationz_new, json_file)
      
%cd /root/Mask_RCNN/dataset/train
traductionANNOTATE("annotate_region_data.csv")

%cd /root/Mask_RCNN/dataset/val
traductionANNOTATE("annotate_region_data.csv")

**4. Préparation et configuration de l'algorithme**

In [ ]:
%cd /root/Mask_RCNN/

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob

ROOT_DIR = os.path.abspath("/")

import warnings
warnings.filterwarnings("ignore")

sys.path.append(ROOT_DIR)  
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import illustration

IMAGE_DIR = os.path.join(ROOT_DIR, "dataset")

config = illustration.IllustrationConfig()

class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

**Téléchargement des poids générés sur Droits et Libertés depuis Github**

Télécharge le fichier dans /root/Mask_RCNN/weights/mask_rcnn_illustration_0030.h5

In [ ]:
%cd /root/Mask_RCNN/weights
!wget https://github.com/cyril521/modoap-seg/raw/master/Poids/poidsDL.7z.001
!wget https://github.com/cyril521/modoap-seg/raw/master/Poids/poidsDL.7z.002
!wget https://github.com/cyril521/modoap-seg/raw/master/Poids/poidsDL.7z.003
!7z x /root/Mask_RCNN/weights/poidsDL.7z.001
os.rename("mask_rcnn_illustration_0030.h5", "Poids_Droits_Libertes_120p_30ep.h5")
os.remove("/root/Mask_RCNN/weights/poidsDL.7z.001")
os.remove("/root/Mask_RCNN/weights/poidsDL.7z.002")
os.remove("/root/Mask_RCNN/weights/poidsDL.7z.003")

**5. Lancement de l'entraînement**

L'entraînement peut prendre plusieurs heures en fonction du nombre d'époques choisi. Le résultat est un fichier de poids .h5 sauvegardé dans */root/Mask_RCNN/weights/illustration* (seul le dernier fichier h5 généré sera réutilisé par la suite)

Le paramètre --weights= permet de spécifier les poids initiaux à utiliser pour l'entraînement. Les valeurs possibles sont COCO, IMAGENET ou "/chemin/vers/fichier.h5"

La cellule est remplie par défaut avec le chemin vers les poids Droits Et Libertés téléchargés dans la cellule précédente.

In [ ]:
%cd /root/Mask_RCNN/
!python illustration.py train --dataset=dataset/ --weights="/root/Mask_RCNN/weights/Poids_Droits_Libertes_120p_30ep.h5" --log="/root/Mask_RCNN/weights"

**6.1 Téléchargement des poids générés vers le disque dur**

In [ ]:
from google.colab import files
listepoids = [file for file in sorted(glob.glob("/root/Mask_RCNN/weights/*/*.h5"))]
if len(listepoids) == 0 :
    print("Aucun poids généré")
else : 
  poids = listepoids[-1]
  files.download(poids)

**6.2 Téléchargement des poids générés vers un Google Drive**

Nécessite de connecter un compte Google Drive. Le fichier poids est copié à la racine du Drive.

In [ ]:
from google.colab import drive
if not os.path.exists("/content/gdrive"): 
  drive.mount('/content/gdrive') 
%cd /content/gdrive/My Drive

listepoids = [file for file in sorted(glob.glob("/root/Mask_RCNN/weights/*/*.h5"))]
if len(listepoids) == 0 :
    print("Aucun poids généré")
else : 
  poids = listepoids[-1]
  !cp $poids ./
  print("Le poids a été copié sur le Drive")